<link rel='stylesheet' href='../assets/css/main.css'/>

[<< back to main index](../../README.md) 

7.2 : Graphx Shortest Path
============================

### Overview
We will find the shortest path on the graph from one point to another. The lab is done by executing each step
in the Spark shell. <br>This allows the student to examine and understand each step, and to modify parameters as we go.<br>
After you have executed the code in each step individually, you will collect this code in script.

For our data, we will use LinkedIn. Therefore, the shortest path will tell us how to connect to the target person
we want to connect with.

<img src="../assets/images/7.2-connections.png" style="border: 5px solid grey; max-width:100%;" />

### Depends On 
None

### Run time
30 mins


## STEP 1: Start Pyspark with GraphFrames library loaded

**option 1 : Jupyter**

**Note:** Jupyter lab will be already running on the port 8888.
So, kill the process first.
```bash
$ sudo netstat -plnt | grep 8888
The process id will be shown in the output.Replace process id in the kill command

$ sudo kill -9 process id
```

```bash
$ PYSPARK_PYTHON=python3 PYSPARK_DRIVER_PYTHON=jupyter PYSPARK_DRIVER_PYTHON_OPTS=notebook ~/apps/spark/bin/pyspark --packages graphframes:graphframes:0.7.0-spark2.4-s_2.11
```

**option 2 : PySpark (command line)**
```bash
$ PYSPARK_PYTHON=python3  ~/apps/spark/bin/pyspark --packages graphframes:graphframes:0.7.0-spark2.4-s_2.11
```

Replace the version on graphframes with your latest.

## Step 2: Import the GraphFrame libraries

In [2]:
# initialize Spark Session
import os
import sys
top_dir = os.path.abspath(os.path.join(os.getcwd(), "../"))
if top_dir not in sys.path:
    sys.path.append(top_dir)

from init_spark import init_spark
spark = init_spark()
sc = spark.sparkContext

Initializing Spark...
Spark found in :  /home/ubuntu/spark
Spark config:
	 executor.memory=2g
	some_property=some_value
	spark.app.name=TestApp
	spark.files=file:///home/ubuntu/.ivy2/jars/graphframes_graphframes-0.7.0-spark2.4-s_2.11.jar,file:///home/ubuntu/.ivy2/jars/org.slf4j_slf4j-api-1.7.16.jar
	spark.jars=file:///home/ubuntu/.ivy2/jars/graphframes_graphframes-0.7.0-spark2.4-s_2.11.jar,file:///home/ubuntu/.ivy2/jars/org.slf4j_slf4j-api-1.7.16.jar
	spark.master=local[*]
	spark.repl.local.jars=file:///home/ubuntu/.ivy2/jars/graphframes_graphframes-0.7.0-spark2.4-s_2.11.jar,file:///home/ubuntu/.ivy2/jars/org.slf4j_slf4j-api-1.7.16.jar
	spark.sql.warehouse.dir=/tmp/tmp5hb2r5jn
	spark.submit.deployMode=client
	spark.submit.pyFiles=/home/ubuntu/.ivy2/jars/graphframes_graphframes-0.7.0-spark2.4-s_2.11.jar,/home/ubuntu/.ivy2/jars/org.slf4j_slf4j-api-1.7.16.jar
	spark.ui.showConsoleProgress=true
Spark UI running on port 4045


In [3]:
from graphframes import *

## Step 3: Construct the nodes / vertices

This is the graph we are modeling.  In this graph, we have the user name and the number of connections. The number of connections is a natural things to have;  we will store it, but not use it at this time.

<img src="../assets/images/7.2-network.png" style="border: 5px solid grey; max-width:100%;" />

In [5]:
vertices = spark.createDataFrame([
        #direct connections
        (1, "Mark Kerzner", 2757), # (Id, Name, no of connections)
        (2, "Sujee Maniyam", 891),
        (3, "Yaakov Weintraub", 105),
        (4, "Packt Publishing", 2984),
        (5, "Barry Kaufman", 500),
        # indirect connections
        (6, "Tony Piazza", 500),
        (7, "Tim Fox", 500),
        (8, "Vamsi Sistla", 1000)
], ["id", "name", "numconnections"])

vertices.show()

+---+----------------+--------------+
| id|            name|numconnections|
+---+----------------+--------------+
|  1|    Mark Kerzner|          2757|
|  2|   Sujee Maniyam|           891|
|  3|Yaakov Weintraub|           105|
|  4|Packt Publishing|          2984|
|  5|   Barry Kaufman|           500|
|  6|     Tony Piazza|           500|
|  7|         Tim Fox|           500|
|  8|    Vamsi Sistla|          1000|
+---+----------------+--------------+



## Step 4: Initialize connections on the graph
Connections are represented as edges.  Go ahead and create an `edgeArray` to represent the graph data in picture.  
For example, a connection from 
- **Mark Kerzner (1)** 
- to **Sujee Maniyam (2)** 
- is represented as  **(1, 2, 1)**

All connections have a weight of `1`.  Complete the following

In [14]:
edges = spark.createDataFrame([
        (1, 2, 1),
        (1, 3, 1),
        # TODO : add in other edges to represent graph in picture
        (1, 4, 1),
        (1, 5, 1),
        (2, 6, 1),
        (6, 7, 1),
        (3, 7, 1),
        (3, 8, 1),
        (7, 8, 1),
], ["src", "dst", "weight"])

edges.show()

+---+---+------+
|src|dst|weight|
+---+---+------+
|  1|  2|     1|
|  1|  3|     1|
|  1|  4|     1|
|  1|  5|     1|
|  2|  6|     1|
|  6|  7|     1|
|  3|  7|     1|
|  3|  8|     1|
|  7|  8|     1|
+---+---+------+



## Step 5: Create the graph

In [7]:
graph = GraphFrame(vertices, edges)

## Step 6 : Print the graph

In [8]:
graph.vertices.show()

+---+----------------+--------------+
| id|            name|numconnections|
+---+----------------+--------------+
|  1|    Mark Kerzner|          2757|
|  2|   Sujee Maniyam|           891|
|  3|Yaakov Weintraub|           105|
|  4|Packt Publishing|          2984|
|  5|   Barry Kaufman|           500|
|  6|     Tony Piazza|           500|
|  7|         Tim Fox|           500|
|  8|    Vamsi Sistla|          1000|
+---+----------------+--------------+



In [9]:
graph.edges.show()

+---+---+------+
|src|dst|weight|
+---+---+------+
|  1|  2|     1|
|  1|  3|     1|
|  1|  4|     1|
|  1|  5|     1|
|  2|  6|     1|
|  6|  7|     1|
|  3|  7|     1|
|  3|  8|     1|
|  7|  8|     1|
+---+---+------+



## Step 9: Compute shortest distances

<img src="../assets/images/7.2c.png" style="border: 5px solid grey; max-width:100%;" />

Use Pregel to compute shortest distances between the root and every other vertex on the graph. 
Please note that since computing the shortest distance between two vertices anyway involves computing many intermediate short distances,
Pregel takes a generic approach of computing all shortest distances

In [11]:
# hint : landmarks : src, destinations

# from Mark(1) to Sujee(2)
graph.shortestPaths(landmarks=[1,2]).show()

# you can also store the results
# results = graph.shortestPaths(landmarks=[1,2])
# results.show()

+---+----------------+--------------+----------------+
| id|            name|numconnections|       distances|
+---+----------------+--------------+----------------+
|  7|         Tim Fox|           500|              []|
|  6|     Tony Piazza|           500|              []|
|  5|   Barry Kaufman|           500|              []|
|  1|    Mark Kerzner|          2757|[1 -> 0, 2 -> 1]|
|  3|Yaakov Weintraub|           105|              []|
|  8|    Vamsi Sistla|          1000|              []|
|  2|   Sujee Maniyam|           891|        [2 -> 0]|
|  4|Packt Publishing|          2984|              []|
+---+----------------+--------------+----------------+



In [12]:
# from Mark(1) to Vamsi(8)
graph.shortestPaths(landmarks=[1,8]).show()

+---+----------------+--------------+----------------+
| id|            name|numconnections|       distances|
+---+----------------+--------------+----------------+
|  7|         Tim Fox|           500|        [8 -> 1]|
|  6|     Tony Piazza|           500|        [8 -> 2]|
|  5|   Barry Kaufman|           500|              []|
|  1|    Mark Kerzner|          2757|[1 -> 0, 8 -> 2]|
|  3|Yaakov Weintraub|           105|        [8 -> 1]|
|  8|    Vamsi Sistla|          1000|        [8 -> 0]|
|  2|   Sujee Maniyam|           891|        [8 -> 3]|
|  4|Packt Publishing|          2984|              []|
+---+----------------+--------------+----------------+



In [13]:
# from Mark(1) to Everyone
graph.shortestPaths(landmarks=range(1,9)).show(truncate=False)

+---+----------------+--------------+----------------------------------------------------------------+
|id |name            |numconnections|distances                                                       |
+---+----------------+--------------+----------------------------------------------------------------+
|7  |Tim Fox         |500           |[7 -> 0, 8 -> 1]                                                |
|6  |Tony Piazza     |500           |[6 -> 0, 7 -> 1, 8 -> 2]                                        |
|5  |Barry Kaufman   |500           |[5 -> 0]                                                        |
|1  |Mark Kerzner    |2757          |[5 -> 1, 1 -> 0, 6 -> 2, 2 -> 1, 7 -> 2, 3 -> 1, 8 -> 2, 4 -> 1]|
|3  |Yaakov Weintraub|105           |[3 -> 0, 7 -> 1, 8 -> 1]                                        |
|8  |Vamsi Sistla    |1000          |[8 -> 0]                                                        |
|2  |Sujee Maniyam   |891           |[2 -> 0, 6 -> 1, 7 -> 2, 8 -> 3]    

Explain the results
    
## Bonus Lab 1
Construct a small graph of air flights between cities. Use 4-6 cities. Put the prices of flying between two cities into the edges above, replacing the number "1" with the actual price.

Calculate the cheapest flights between cities.